In [ ]:
# 库文件
import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore", category=UserWarning)
warnings.filterwarnings("ignore", category=FutureWarning)
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from skopt import gp_minimize
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error
from scipy.optimize import minimize
from sklearn.metrics.pairwise import rbf_kernel
import pandas as pd
import optuna


In [ ]:
# 从 Excel 文件读取数据
df = pd.read_excel('data.xlsx')

# 提取列数据
u1_data = df['u1'].values
u2_data = df['u2'].values
y1_data = df['y1'].values
y2_data = df['y2'].values

# # 输出提取的数据
# print('u1_data:', u1_data.shape)
# print('u2_data:', u2_data.shape)
# print('y1_data:', y1_data.shape)
# print('y2_data:', y1_data.shape)

num_samples = y1_data.shape[0]


In [ ]:
# 组合训练数据  多输入双输出
# 假设有训练数据 u1_data, u2_data, y1_data 和 y1_data+1
# 将它们组合成输入特征矩阵 X 和输出标签向量 y
u1_data = u1_data
u2_data = u2_data

u1_data_1 = np.roll(u1_data, 1)
u2_data_1 = np.roll(u2_data, 1)
u1_data_1[0], u2_data_1[0] = 0, 0

y1_data = y1_data
y2_data = y2_data

X = np.column_stack((u1_data[:-1], u2_data[:-1], u1_data_1[:-1], u2_data_1[:-1], y1_data[:-1], y2_data[:-1]))
y = np.column_stack((y1_data[1:],y2_data[1:]))

# print(X.shape,y.shape)

# 划分训练集和测试集
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3333, random_state=42)


In [ ]:
# 获取模型参数
def get_params(W_b):

    mid_indix = W_b.shape[0]//2-1
    pred_0 = W_b[-2:-1]
    W0 = W_b[:mid_indix]
    b1 = W_b[-1:]
    W1 = W_b[mid_indix:mid_indix*2]
    W_b_0 = np.concatenate((W0, pred_0))
    W_b_1 = np.concatenate((W1, b1))
    # print(mid_indix)

    # print('pred_0:',pred_0.shape)
    # print('W0:',W0.shape)
    # print('b1:',b1.shape)
    # print('W1:',W1.shape)
    return pred_0,W0,b1,W1,W_b_0,W_b_1


In [ ]:
# 定义My_LS_SVRModel
class My_M_LS_SVRModel:
    def __init__(self, params):
        C0, C1, C00, gamma= params
        self.C0 = C0
        self.C1 = C1
        self.C00 = C00
        self.gamma = gamma
        self.W_b = None
        self.X_train = None

    def model_train(self, X_train, y_train, K_train):
        def objective(W_b, X, y):
            b0,W0,b1,W1,W_b_0,W_b_1 = get_params(W_b)

            y_pred_0 = np.dot(K_train, W0) + b0
            y_pred_1 = np.dot(K_train, W1) + b1
            errors0 = y[:,0] - y_pred_0
            errors1 = y[:,1] - y_pred_1

            # 损失函数
            loss = ( 0.5 * (np.dot(W0, W0)+np.dot(W1, W1)) 
                    + self.C0 * np.sum(errors0**2) + self.C1 * np.sum(errors1**2)
                    # + C00 * np.sum(np.sqrt(errors0**2 + errors1**2)) # L2 范数
                    + self.C00 * np.sum(errors0**2 + errors1**2) # L2 范数的平方
                    )
            return loss

        # 初始化权重向量+偏移项b
        initial_W_b = np.zeros((X_train.shape[0])*2+(1)*2)

        # 使用minimize 函数最小化目标函数
        result = minimize(objective, initial_W_b, args=(X_train, y_train),
                            method='L-BFGS-B')
        # 输出最优的权重向量
        best_W_b = result.x
        return best_W_b

    def fit(self, X_train, y_train):
        self.X_train = X_train
        K_train = rbf_kernel(X_train, X_train, gamma=self.gamma)
        self.W_b = self.model_train(X_train, y_train, K_train)

    def predict(self, X_test):
        b0,W0,b1,W1,W_b_0,W_b_1 = get_params(self.W_b)
        K_test = rbf_kernel(X_test, self.X_train, gamma=self.gamma)
        y_pred_0 = np.dot(K_test, W0) + b0
        y_pred_1 = np.dot(K_test, W1) + b1
        return y_pred_0, y_pred_1
    

# # 示例用法
# params = [8.78525340e+01, 2.01347249e-03]
# # 创建模型
# my_svr_model = My_LS_SVRModel(params=params)
# # 训练模型
# my_svr_model.fit(X_train, y_train)
# # 模型预测
# y_pred = my_svr_model.predict(X_test)


In [ ]:
def your_fitness_function(trial, params, X_train, y_train, X_test, y_test):
    # 示例用法
    # 创建模型
    my_svr_model = My_M_LS_SVRModel(params=params)
    # 训练模型
    my_svr_model.fit(X_train, y_train)
    # 模型预测
    y_pred_0, y_pred_1 = my_svr_model.predict(X_test)        
    # 在这里计算适应值（均方根误差）
    errors0 = y_test[:,0] - y_pred_0
    errors1 = y_test[:,1] - y_pred_1
    mse = np.sqrt(np.sum(errors0**2+errors1**2)/X_test.shape[0])

    return mse


# 目标函数适配 Optuna 风格
def objective(trial, X_train, y_train, X_test, y_test):
    # 在搜索空间中定义超参数
    C0 = trial.suggest_float('C0', 1e-6, 20.0, log=True)
    C1 = trial.suggest_float('C1', 1e-6, 20.0, log=True)
    C00 = trial.suggest_float('C00', 1e-6, 20.0, log=True)
    gamma = trial.suggest_float('gamma', 1e-6, 1.0, log=True)

    # 调用你的目标函数
    params = [C0 ,C1, C00, gamma]
    mse = your_fitness_function(trial, params, X_train, y_train, X_test, y_test)

    # 返回目标值（Optuna 默认是最小化目标）
    return mse

# 创建 Optuna 试验对象
study = optuna.create_study(direction='minimize')

# 执行优化
study.optimize(lambda trial: objective(trial, X_train, y_train, X_test, y_test), n_trials=100)

# 输出结果
print(f"Best Trial: {study.best_trial.params}")
print(f"Best MSE: {study.best_value}")

# 绘制优化过程
optuna.visualization.plot_optimization_history(study)



In [ ]:
# 创建模型
#  data  [23.65353603, 46.24063272, 14.1795326,   0.14215454]  0.29159782217670577
#  data2  [36.82522053 53.21374946 18.48987828  0.15809682]  0.5322094851234387
params = [36.82522053 ,53.21374946 ,18.48987828,  0.15809682] 
my_svr_model = My_M_LS_SVRModel(params=params)
# 训练模型
my_svr_model.fit(X_train, y_train)
# 模型预测
y_pred_0, y_pred_1 = my_svr_model.predict(X_test)


In [ ]:
# 计算 RMSE
y_test = y_test
rmse_0 = np.sqrt(mean_squared_error(y_test[:, 0], y_pred_0))
rmse_1 = np.sqrt(mean_squared_error(y_test[:, 1], y_pred_1))

# 计算 MRE
mre_0 = np.mean(np.abs((y_test[:, 0] - y_pred_0) / y_test[:, 0])) * 100
mre_1 = np.mean(np.abs((y_test[:, 1] - y_pred_1) / y_test[:, 1])) * 100

# 打印结果
print("Dimension 0:")
print("RMSE:", rmse_0)
print("MRE:", mre_0)

print("\nDimension 1:")
print("RMSE:", rmse_1)
print("MRE:", mre_1)


In [ ]:
# 模型预测结果
# 创建两个子图，分别绘制每个维度
plt.figure(figsize=(12, 6))

# 第一个维度的曲线
plt.subplot(2, 1, 1)
plt.plot(y_test[:, 0], 'r-', label='y_test')
plt.plot(y_pred_0, 'b-', label='y_pred')
plt.xlabel('Sample Index')
plt.ylabel('Value')
plt.title('Comparison of y_test and y_pred (Dimension 0)')
plt.legend()

# 第二个维度的曲线
plt.subplot(2, 1, 2)
plt.plot(y_test[:, 1], 'r-', label='y_test')
plt.plot(y_pred_1, 'b-', label='y_pred')
plt.xlabel('Sample Index')
plt.ylabel('Value')
plt.title('Comparison of y_test and y_pred (Dimension 1)')
plt.legend()

# 调整子图布局
plt.tight_layout()
plt.show()
